#EDA

### Importaciones

In [1]:
import pandas as pd
import json
import numpy as np
import pickle
import pyarrow.parquet as pq
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Yelp

procedemos a extraer informacion relevante sobre las categorias de entrenimientos planteadas en nuetro 3er kpi

In [ ]:
archivo_pkl = '/content/drive/MyDrive/Data/Yelp/business.pkl'
with open(archivo_pkl, 'rb') as archivo:
    business = pickle.load(archivo)
business = business.loc[:, ~business.columns.duplicated()]
business = business.drop(columns=['attributes'])
business = business.drop(columns=['hours'])

In [ ]:
business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma..."


procedo a normalizar las cateogries, para un mejor manejo de este dataframe

In [ ]:
# Reemplazar todo el valor por 'Arte y Entretenimiento' si contiene la palabra 'Arts'
business['categories'] = business['categories'].apply(lambda x: 'Arte y Entretenimiento' if 'Arts' in x else x)

In [ ]:
# Eliminar las filas con valores NaN en la columna 'categories'
business = business.dropna(subset=['categories'])
filas_arts = business[business['categories'].str.contains('Arte y Entretenimiento')]

# Mostrar las filas filtradas
filas_arts.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories


In [ ]:
# Reemplazar todo el valor por 'Shopping' si contiene la palabra 'Shopping'
business['categories'] = business['categories'].apply(lambda x: 'Shopping' if 'Shopping' in x else x)

Selecciono las categorias de yelp que vamos a utilizar

In [ ]:
categorias = ['Shopping', 'Banks', 'Arts', 'Nightlife']
resultados = pd.DataFrame()

In [ ]:
for categoria in categorias:
    # Filtrar el DataFrame original por cada categoría
    df_categoria = business[business['categories'].str.contains(categoria, case=False, na=False)]

    # Agregar una nueva columna para identificar la categoría
    df_categoria['categoria'] = categoria

    # Concatenar los resultados al DataFrame final
    resultados = pd.concat([resultados, df_categoria], ignore_index=True)

    # Eliminar la columna 'categories'
    resultados = resultados.drop(columns=['categories'])

    # Reemplazar los valores en la columna 'categoria'
    resultados['categoria'] = resultados['categoria'].replace({
          'Nightlife': 'Vida nocturna',
          'Arts': 'Arte y Entretenimiento',
          'Banks': 'Servicios Financieros',
          'Shopping': 'Shopping'
    })

# Mostrar las primeras filas del DataFrame final
resultados

<ipython-input-9-3bdc9235eec9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_categoria['categoria'] = categoria
<ipython-input-9-3bdc9235eec9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_categoria['categoria'] = categoria
<ipython-input-9-3bdc9235eec9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categoria
0,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,Shopping
1,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,PA,63144,38.627695,-90.340465,2.5,13,1,Shopping
2,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,1,Shopping
3,M0XSSHqrASOnhgbWDJIpQA,Herb Import Co,712 Adams St,New Orleans,FL,70118,29.941468,-90.129953,4.0,5,1,Shopping
4,qhDdDeI3K4jy2KyzwFN53w,Barnes & Noble Booksellers,8675 River Crossing Blvd,Indianapolis,LA,46240,39.91382,-86.105739,4.0,38,1,Shopping
...,...,...,...,...,...,...,...,...,...,...,...,...
34750,9iCum5W48TqNVDTZCJUKjg,Plantation Pub,8321 Sawyer Brown Rd,Bellevue,FL,37221,36.070748,-86.947891,3.5,40,1,Vida nocturna
34751,IeSD0nMKRFYUTnR5nZH1CQ,HighWire Lounge,14 S Arizona Ave,Tucson,CA,85701,32.221828,-110.967969,3.5,111,1,Vida nocturna
34752,-R9t30RvsG-Rtiq63yuEmg,Crescent No Lawyers Bar-Grill,5500 W Franklin Rd,Boise,IN,83706,43.603368,-116.250063,2.5,55,0,Vida nocturna
34753,esBGrrmuZzSiECyRBoKvvA,Colony Grill - St. Petersburg,670 Central Ave,St. Petersburg,FL,33701,27.770872,-82.643069,4.5,38,1,Vida nocturna


al realizar ciertas pruebas de los datos, descubrimo que las columnas state y city contienen datos errones, debito a esto procedemos a utilizar la libreria reverse_geocoder para a traves de la longitud y la latitud poder averiguar la ciudad y el estado correcto de los negocios

In [ ]:
import reverse_geocoder as rg

def obtener_estado_y_ciudad(lat, lon):
    coordinates = (lat, lon)
    result = rg.search(coordinates)  # Realiza la búsqueda inversa
    estado = result[0]['admin1']  # Obtiene el estado de la respuesta
    ciudad = result[0]['name']  # Obtiene el nombre de la ciudad de la respuesta
    return estado, ciudad
def obtener_estado_y_ciudad(lat, lon):
    coordinates = (lat, lon)
    result = rg.search(coordinates)  # Realiza la búsqueda inversa
    estado = result[0]['admin1']  # Obtiene el estado de la respuesta
    ciudad = result[0]['name']  # Obtiene el nombre de la ciudad de la respuesta
    return estado, ciudad

# Aplica la función a cada fila del DataFrame
resultados[['estado', 'ciudad']] = resultados.apply(lambda x: pd.Series(obtener_estado_y_ciudad(x['latitude'], x['longitude'])), axis=1)

Loading formatted geocoded file...


In [ ]:
resultados.head(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categoria,estado,ciudad
0,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,Shopping,Arizona,Tucson
1,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,PA,63144,38.627695,-90.340465,2.5,13,1,Shopping,Missouri,Brentwood
2,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.45738,3.5,6,1,Shopping,Florida,Land O' Lakes
3,M0XSSHqrASOnhgbWDJIpQA,Herb Import Co,712 Adams St,New Orleans,FL,70118,29.941468,-90.129953,4.0,5,1,Shopping,Louisiana,Jefferson
4,qhDdDeI3K4jy2KyzwFN53w,Barnes & Noble Booksellers,8675 River Crossing Blvd,Indianapolis,LA,46240,39.91382,-86.105739,4.0,38,1,Shopping,Indiana,Meridian Hills
5,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,NV,93101,34.420209,-119.70046,3.0,24,1,Shopping,California,Santa Barbara
6,mGFsPRHdE6SMV1gYQ6GzgQ,T-Mobile,4735 Concord Pike,Wilmington,LA,19803,39.822176,-75.546007,2.0,6,1,Shopping,Delaware,Bellefonte
7,x1mhq4IpWctQOBM06dU8vg,Diamond Hair Company,5324 W 16th St,Indianapolis,DE,46224,39.787464,-86.251421,1.5,25,1,Shopping,Indiana,Speedway
8,mglME3E7esPSpIF5bias8w,Pier 1,1014 Baltimore Pike,Springfield,IL,19064,39.91527,-75.342681,2.5,8,0,Shopping,Pennsylvania,Swarthmore
9,txyXRytGjwOXvS8s4sc-WA,Smoothie King,1070 E Tucson Marketplace Blvd,Tucson,AB,85713,32.186794,-110.954765,3.0,29,1,Shopping,Arizona,South Tucson


In [ ]:
resultados.to_csv('/content/drive/My Drive/Colab Notebooks/business-entretenimiento-filtrado.csv', index=False, encoding='utf-8')

In [ ]:
resultados = pd.read_csv('/content/drive/My Drive/Colab Notebooks/business-entretenimiento-filtrado.csv')

In [ ]:
unicos = resultados["estado"].unique()
unicos

array(['Arizona', 'Missouri', 'Florida', 'Louisiana', 'Indiana',
       'California', 'Delaware', 'Pennsylvania', 'Tennessee', 'Alberta',
       'Nevada', 'New Jersey', 'Idaho', 'Illinois'], dtype=object)

una vez filtrado por categorias procedemos  a filtrarlos por los estados que vamos a utilizar

In [ ]:
# Lista de estados a filtrar
estados_filtrar = ['Arizona', 'California', 'Nevada']

# Filtrar el DataFrame para incluir solo los estados deseados
resultados = resultados[resultados['estado'].isin(estados_filtrar)]

mostramos como es el resultado

In [ ]:
resultados.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categoria,estado,ciudad
0,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,Shopping,Arizona,Tucson
5,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,NV,93101,34.420209,-119.700460,3.0,24,1,Shopping,California,Santa Barbara
9,txyXRytGjwOXvS8s4sc-WA,Smoothie King,1070 E Tucson Marketplace Blvd,Tucson,AB,85713,32.186794,-110.954765,3.0,29,1,Shopping,Arizona,South Tucson
12,eYxGFkxo6m3SYGVTh5m2nQ,Big Boyz Toyz Motorcycle Rentals,4158 E Grant Rd,Tucson,PA,85712,32.250324,-110.903655,4.5,8,1,Shopping,Arizona,Tucson
17,knQ4vIgx-r85kjlWVVjcpQ,Silver and Blue Outfitters,5191 Meadowood Mall Cir,Reno,PA,89502,39.476518,-119.784037,4.5,5,1,Shopping,Nevada,Reno


In [ ]:
# Renombrar la columna 'categoria' a 'categories'
resultados = resultados.rename(columns={'categoria': 'categories'})

In [ ]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5388 entries, 0 to 34751
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   5388 non-null   object 
 1   name          5388 non-null   object 
 2   address       5266 non-null   object 
 3   postal_code   5386 non-null   object 
 4   latitude      5388 non-null   float64
 5   longitude     5388 non-null   float64
 6   stars         5388 non-null   float64
 7   review_count  5388 non-null   int64  
 8   categories    5388 non-null   object 
 9   state         5388 non-null   object 
 10  city          5388 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 505.1+ KB


In [ ]:
columnas_a_eliminar = ['is_open', 'city', 'state']

# Elimina las columnas especificadas
resultados = resultados.drop(columnas_a_eliminar, axis=1)

In [ ]:
resultados = resultados.rename(columns={'estado': 'state','ciudad': 'city'})

In [ ]:
resultados = resultados.rename(columns={'stars': 'business_stars'})

In [ ]:
resultados.to_csv('/content/drive/My Drive/Colab Notebooks/business-entre-estados-necesitados.csv', index=False, encoding='utf-8')

una vez terminado con los negocios que vamos a utilizar, procedemos a abrir y filtrar nuestros datos de reviews

In [2]:
review = '/content/drive/MyDrive/Data/Yelp/review-002.json'
rows_to_read = 5000000  # Número total de filas que deseas leer
chunk_size = 1000000  # Tamaño del fragmento

def limpiar_datos(line):
    try:
        line = line.replace('\\', ' ')
        return json.loads(line)
    except json.JSONDecodeError:
        return None

# Inicializar el DataFrame
df_review = pd.DataFrame()

# Crear un generador para cargar el archivo en fragmentos
chunks = pd.read_json(review, lines=True, chunksize=chunk_size, nrows=rows_to_read)

# Iterar sobre los fragmentos y procesar cada uno
for chunk in chunks:
    datos = [limpiar_datos(line) for line in chunk.to_json(orient='records', lines=True).split('\n') if line.strip()]
    datos = [d for d in datos if d is not None]  # Eliminar datos nulos
    df_temp = pd.DataFrame(datos)

    if 'user_id' in df_temp.columns:
        df_temp.drop('user_id', axis=1, inplace=True)
    if 'cool' in df_temp.columns:
        df_temp.drop('cool', axis=1, inplace=True)
    if 'funny' in df_temp.columns:
        df_temp.drop('funny', axis=1, inplace=True)
    if 'useful' in df_temp.columns:
        df_temp.drop('useful', axis=1, inplace=True)

    # Rellenar valores nulos con ceros
    df_temp.fillna(0, inplace=True)

    # Concatenar el fragmento al DataFrame principal
    df_review = pd.concat([df_review, df_temp], ignore_index=True)

    # Verificar si ya se han leído las filas deseadas
    if len(df_review) >= rows_to_read:
        break

In [3]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4421848 entries, 0 to 4421847
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_id    object
 1   business_id  object
 2   stars        int64 
 3   text         object
 4   date         int64 
dtypes: int64(2), object(3)
memory usage: 168.7+ MB


la columna "date" parecen ser representaciones de fechas en formato de tiempo UNIX, que es el número de milisegundos transcurridos desde la medianoche UTC del 1 de enero de 1970. se procede a realizar una transformacion en estos

In [4]:
df_review['date'].head()

0    1531001351000
1    1325604498000
2    1391632230000
3    1420329663000
4    1484427255000
Name: date, dtype: int64

In [5]:
# Convertir la columna 'date' de milisegundos a segundos y luego a formato de fecha
df_review['date'] = pd.to_datetime(df_review['date'] // 1000, unit='s')

# Mostrar las primeras filas de la columna 'date'
df_review['date'].head()

0   2018-07-07 22:09:11
1   2012-01-03 15:28:18
2   2014-02-05 20:30:30
3   2015-01-04 00:01:03
4   2017-01-14 20:54:15
Name: date, dtype: datetime64[ns]

In [6]:
df_review = df_review.rename(columns={'stars': 'review_stars'})

In [7]:
df_review.head()

,review_id,business_id,review_stars,text,date
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


procedemos a realizar el merge entre el dataframe de nuestros negocios y las reviews

In [8]:
# Ruta del archivo CSV en Google Drive
ruta_archivo_csv = '/content/drive/My Drive/Colab Notebooks/business-entre-estados-necesitados.csv'

# Lee el archivo CSV usando pandas
resultados = pd.read_csv(ruta_archivo_csv)

In [9]:
#combinamos desde la tabla businnes los reviwes q tenga el mismo business_id de ellos
combined_df_business_reviwes = df_review.merge(resultados, on='business_id', how='inner')


In [10]:
combined_df_business_reviwes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190093 entries, 0 to 190092
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   review_id       190093 non-null  object        
 1   business_id     190093 non-null  object        
 2   review_stars    190093 non-null  int64         
 3   text            190093 non-null  object        
 4   date            190093 non-null  datetime64[ns]
 5   name            190093 non-null  object        
 6   address         188709 non-null  object        
 7   postal_code     190092 non-null  float64       
 8   latitude        190093 non-null  float64       
 9   longitude       190093 non-null  float64       
 10  business_stars  190093 non-null  float64       
 11  review_count    190093 non-null  int64         
 12  categories      190093 non-null  object        
 13  state           190093 non-null  object        
 14  city            190093 non-null  obj

In [11]:
combined_df_business_reviwes.head(2)

,review_id,business_id,review_stars,text,date,name,address,postal_code,latitude,longitude,business_stars,review_count,categories,state,city
0,xdQe2E8jR6pLA07p1aYtKA,X8lfGPagfLzfOsx0k08NRQ,5,Took my vehicle here for some work a few years...,2017-05-13 14:15:29,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,29,Arte y Entretenimiento,Nevada,Reno
1,s5dCT9uQeAW9KzTGdq8hIg,X8lfGPagfLzfOsx0k08NRQ,4,Brought my 4Runner to replace catalyst convert...,2016-02-23 18:56:59,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,29,Arte y Entretenimiento,Nevada,Reno


Como próximo paso, llevamos a cabo un análisis de sentimiento sobre las reseñas que los usuarios proporcionaron, con el objetivo de compararlo posteriormente con las calificaciones en estrellas que estos mismos usuarios otorgan al negocio. De esta manera, buscamos determinar cuál de estos criterios es más fidedigno para nuestra evaluación

In [12]:
from textblob import TextBlob
# Función para asignar categorías personalizadas
def categorizar_sentimiento(polaridad):
    if polaridad > 0.5:
        return 'Excelente'
    elif 0.1 <= polaridad <= 0.5:
        return 'Bueno'
    elif -0.1 <= polaridad < 0.1:
        return 'Regular'
    elif -0.5 <= polaridad < -0.1:
        return 'Malo'
    else:
        return 'Muy Malo'

# Aplica el análisis de sentimiento solo para reseñas presentes
mask = combined_df_business_reviwes['text'].notnull()
combined_df_business_reviwes.loc[mask, 'sentiment_analysis'] = combined_df_business_reviwes.loc[mask, 'text'].apply(lambda x: categorizar_sentimiento(TextBlob(str(x)).sentiment.polarity))

combined_df_business_reviwes.head()

,review_id,business_id,review_stars,text,date,name,address,postal_code,latitude,longitude,business_stars,review_count,categories,state,city,sentiment_analysis
0,xdQe2E8jR6pLA07p1aYtKA,X8lfGPagfLzfOsx0k08NRQ,5,Took my vehicle here for some work a few years...,2017-05-13 14:15:29,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,29,Arte y Entretenimiento,Nevada,Reno,Bueno
1,s5dCT9uQeAW9KzTGdq8hIg,X8lfGPagfLzfOsx0k08NRQ,4,Brought my 4Runner to replace catalyst convert...,2016-02-23 18:56:59,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,29,Arte y Entretenimiento,Nevada,Reno,Bueno
2,rFo9Tnob93KeSDUZc_s40A,X8lfGPagfLzfOsx0k08NRQ,5,Let these guys do my Borla Catback exhaust sys...,2016-12-26 04:20:21,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,29,Arte y Entretenimiento,Nevada,Reno,Bueno
3,-V2PboBEGq11aZMyaYBioQ,X8lfGPagfLzfOsx0k08NRQ,5,My muffler came loose from the grommets that h...,2017-08-09 14:55:15,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,29,Arte y Entretenimiento,Nevada,Reno,Bueno
4,Fh3vB2d2c57ejdk1R0rZMA,X8lfGPagfLzfOsx0k08NRQ,5,Had a great experience at Landa Muffler. They ...,2014-03-19 19:41:21,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,29,Arte y Entretenimiento,Nevada,Reno,Bueno


Categorizamos la columna "review_stars" asignando las etiquetas de 1 como "Muy Malo", 2 como "Malo", 3 como "Regular", y 4 y 5 como "Bueno" y "Excelente", respectivamente. Posteriormente, comparamos esta clasificación con nuestro análisis de sentimiento, donde aplicamos también una categorización. En este caso, asignamos las siguientes etiquetas: 0 representa un comentario negativo, 1 un comentario neutro y 2 un comentario positivo.

In [14]:
# Define las categorías para las estrellas
star_mapping = {1: 'Muy Malo', 2: 'Malo', 3: 'Regular', 4: 'Bueno', 5: 'Excelente'}
combined_df_business_reviwes['stars_category'] = combined_df_business_reviwes['review_stars'].map(star_mapping)

In [16]:
# Selecciona las columnas que deseas mostrar
selected_columns = ['stars_category', 'sentiment_analysis','review_stars','text']

# Muestra las primeras 20 filas de las columnas seleccionadas
combined_df_business_reviwes[selected_columns].head(5)

,stars_category,sentiment_analysis,review_stars,text
0,Excelente,Bueno,5,Took my vehicle here for some work a few years...
1,Bueno,Bueno,4,Brought my 4Runner to replace catalyst convert...
2,Excelente,Bueno,5,Let these guys do my Borla Catback exhaust sys...
3,Excelente,Bueno,5,My muffler came loose from the grommets that h...
4,Excelente,Bueno,5,Had a great experience at Landa Muffler. They ...


Como se observan rápidamente algunas discrepancias entre los dos análisis, creamos una tabla de contingencias para evaluarlas.

In [17]:
# Crea la tabla de contingencia
contingency_table = pd.crosstab(combined_df_business_reviwes['sentiment_analysis'], combined_df_business_reviwes['stars_category'], margins=True, margins_name='Total')

# Renombra la columna 'All' a 'Total'
contingency_table.rename(columns={'All': 'Total'}, inplace=True)

# Visualiza la tabla de contingencia
print(contingency_table)

stars_category      Bueno  Excelente   Malo  Muy Malo  Regular   Total
sentiment_analysis                                                    
Bueno               28724      70640   5513      5468    11284  121629
Excelente            4557      22761    152       188      607   28265
Malo                  258        422   1622      7624      601   10527
Muy Malo               12         35     76       756       17     896
Regular              2928       4446   5600     11925     3877   28776
Total               36479      98304  12963     25961    16386  190093


se realiza un mapeo de las categorías de la columna "sentiment_analysis" a números del 1 al 5, asignándoles un valor numérico acorde a su grado de positividad. Luego, se asignan pesos a las métricas, especificando que la métrica de las estrellas de la revisión tiene un peso del 70%, mientras que el análisis de sentimiento tiene un peso del 30%. Finalmente, se calcula una nueva métrica combinada, donde se promedian ponderadamente los valores de las métricas de las estrellas y el análisis de sentimiento, obteniendo así una medida combinada que refleja tanto la evaluación de los usuarios como el análisis de sentimiento, considerando sus respectivos pesos. Este enfoque permite integrar ambas fuentes de información de manera equilibrada en una sola métrica para evaluar la satisfacción general

In [18]:
# Mapeo de categorías a números del 1 al 5
sentiment_mapping_numerico = {'Excelente': 5, 'Bueno': 4, 'Regular': 3, 'Malo': 2, 'Muy Malo': 1}

# Aplicar el mapeo a la columna sentiment_analysis
combined_df_business_reviwes['sentiment_analysis_numerico'] = combined_df_business_reviwes['sentiment_analysis'].map(sentiment_mapping_numerico)

# Asignar pesos a las métricas
peso_review_stars = 0.7
peso_analisis_sentimiento = 0.3

# Calcular métrica combinada
combined_df_business_reviwes['metrica_combinada'] = (peso_review_stars * combined_df_business_reviwes['review_stars'] +
                           peso_analisis_sentimiento * combined_df_business_reviwes['sentiment_analysis_numerico']) / (peso_review_stars + peso_analisis_sentimiento)

In [19]:
selected_columns = ['stars_category', 'sentiment_analysis','review_stars','metrica_combinada']

combined_df_business_reviwes[selected_columns].head(5)

,stars_category,sentiment_analysis,review_stars,metrica_combinada
0,Excelente,Bueno,5,4.7
1,Bueno,Bueno,4,4.0
2,Excelente,Bueno,5,4.7
3,Excelente,Bueno,5,4.7
4,Excelente,Bueno,5,4.7


In [20]:
columnas_a_eliminar = ['sentiment_analysis_numerico', 'stars_category', 'sentiment_analysis','review_count','text','review_stars']

combined_df_business_reviwes.drop(columnas_a_eliminar, axis=1, inplace=True)

In [21]:
nombres_nuevos = {'metrica_combinada': 'review_stars'}

combined_df_business_reviwes.rename(columns=nombres_nuevos, inplace=True)

In [22]:
combined_df_business_reviwes.head()

,review_id,business_id,date,name,address,postal_code,latitude,longitude,business_stars,categories,state,city,review_stars
0,xdQe2E8jR6pLA07p1aYtKA,X8lfGPagfLzfOsx0k08NRQ,2017-05-13 14:15:29,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,Arte y Entretenimiento,Nevada,Reno,4.7
1,s5dCT9uQeAW9KzTGdq8hIg,X8lfGPagfLzfOsx0k08NRQ,2016-02-23 18:56:59,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,Arte y Entretenimiento,Nevada,Reno,4.0
2,rFo9Tnob93KeSDUZc_s40A,X8lfGPagfLzfOsx0k08NRQ,2016-12-26 04:20:21,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,Arte y Entretenimiento,Nevada,Reno,4.7
3,-V2PboBEGq11aZMyaYBioQ,X8lfGPagfLzfOsx0k08NRQ,2017-08-09 14:55:15,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,Arte y Entretenimiento,Nevada,Reno,4.7
4,Fh3vB2d2c57ejdk1R0rZMA,X8lfGPagfLzfOsx0k08NRQ,2014-03-19 19:41:21,Landa Muffler & Brake,816 E 4th St,89512.0,39.531787,-119.802696,3.5,Arte y Entretenimiento,Nevada,Reno,4.7


una vez finalizado el proceso se procede a guardar el archvo en un csv

In [23]:
combined_df_business_reviwes.to_csv('/content/drive/My Drive/Colab Notebooks/yelp-review-business-entretenimiento.csv', index=False, encoding='utf-8')